In [1]:
import pandas as pd
import tools as t

In [178]:
df_descr = pd.DataFrame(pd.read_json('deduplicated.json')[['id', 'description']])

Разделение на предложения

In [4]:
from nltk import sent_tokenize

In [179]:
df_descr['sent_tok'] = df_descr['description'].apply(lambda x: sent_tokenize(x, language = 'russian'))

In [180]:
not_to_split = ['a/b', 'ci/cd', 'pl/sql', 'pl/pg', 'tcp/ip', 'ui/ux', 'ux/ui', 'а/б', 'а/в', 'в/из']

Очистим от ненужных символов

In [181]:
def clean_list(x):
    cleaned_list = []
    for one in x:
        cleaned_list.append(t.clean_text(one, not_to_split))
    return cleaned_list

In [182]:
df_descr['sent_tok'] = df_descr['sent_tok'].apply(lambda x: clean_list(x))

токенизация по словам

In [35]:
from nltk import word_tokenize

In [36]:
def word_tok(sent_tok):
    tokenized = []
    for one in sent_tok:
        tokenized.append(word_tokenize(one, language = 'russian')) 
    return(tokenized)

In [183]:
df_descr['word_tok'] = df_descr['sent_tok'].apply(lambda x: word_tok(x))

Лемматизация

In [53]:
def lemm(word_tok):
    lemmatized = []
    for one in word_tok:
        lemmatized.append(t.lemmatization(one))
    return lemmatized

In [184]:
df_descr['word_tok'] = df_descr['word_tok'].apply(lambda x: lemm(x))

Удаление стоп-слов

In [185]:
from nltk.corpus import stopwords
sw_to_add = ['хорошее', 'знание', 'знать', 'опыт', 'работы', 'организация', 'работать', 'ms', 'условия', 'умение', 'навык']
sw_to_add = t.lemmatization(sw_to_add)
stopwords = stopwords.words('russian')
stopwords = stopwords + sw_to_add

In [186]:
def sw(word_tok):
    res = []
    for one in word_tok:
        res.append(t.remove_stop_words(one, stopwords))
    return res

In [187]:
df_descr['wo_sw'] = df_descr['word_tok'].apply(lambda x: sw(x))

In [188]:
df_descr.to_json('dataset_to_train.json')

Создание двумерного списка 

In [189]:
temp_list = df_descr['wo_sw'].to_list()

In [190]:
descr_list = []
for one_text in temp_list:
    for one_sent in one_text:
        descr_list.append(one_sent)

In [191]:
descr_list

[['связь',
  'развитие',
  'сеть',
  'структура',
  'отдел',
  'компания',
  'рассмотреть',
  'специалист',
  'отдел',
  'аналитик'],
 ['проект',
  'задача',
  'автоматизация',
  'оптимизация',
  'процесс',
  'структура',
  'цепочка',
  'закуп',
  'управление',
  'товарный',
  'запас'],
 ['открыть', 'обсуждение', 'задача', 'высоко', 'компетентный', 'специалист'],
 ['уровень',
  'заработный',
  'плата',
  'обсуждаться',
  'результат',
  'оценка',
  'компетенция'],
 ['просить', 'указывать', 'резюме', 'ваш', 'финансовый', 'ожидание'],
 ['обязанность',
  'сбор',
  'обработка',
  'анализ',
  'больший',
  'объём',
  'данные',
  'excel',
  'анализ',
  'тренд',
  'прогнозирование',
  'участие',
  'автоматизация',
  'процесс',
  'тестирование',
  'система',
  'анализ',
  'причина',
  'отклонение',
  'плановый',
  'показатель',
  'аналитический',
  'отчётность',
  'требование',
  'excel',
  'уровень',
  'продвинуть',
  'пользователь',
  'впр',
  'сводный',
  'таблица',
  'еслимн',
  'др',
  'фун

выделение наиболее значимыз биграмм в тексте

In [62]:
from gensim.models.phrases import Phrases, Phraser

Обучение модели

In [193]:
def build_phrases(sentences):
    phrases = Phrases(sentences,
                      min_count=5,
                      threshold=7,
                      progress_per=1000)
    return phrases

для биграмм

In [194]:
bigrams = build_phrases(descr_list)

In [195]:
trigrams = build_phrases(bigrams[descr_list])

Предложения, в которых важны биграммы заменены на одно слово (объеденены в одно слово)

In [196]:
res = []
for one in descr_list:
    res.append(trigrams[bigrams[one]])

In [197]:
for i in range(100):
    print(res[i])

['связь', 'развитие', 'сеть', 'структура', 'отдел', 'компания', 'рассмотреть', 'специалист', 'отдел', 'аналитик']
['проект', 'задача', 'автоматизация', 'оптимизация', 'процесс', 'структура', 'цепочка', 'закуп', 'управление_товарный_запас']
['открыть', 'обсуждение', 'задача', 'высоко', 'компетентный_специалист']
['уровень_заработный_плата', 'обсуждаться_результат', 'оценка', 'компетенция']
['просить_указывать_резюме', 'ваш', 'финансовый', 'ожидание']
['обязанность_сбор_обработка', 'анализ', 'больший_объём_данные', 'excel', 'анализ', 'тренд_прогнозирование', 'участие', 'автоматизация_процесс', 'тестирование', 'система', 'анализ_причина_отклонение', 'плановый_показатель', 'аналитический_отчётность', 'требование', 'excel_уровень_продвинуть', 'пользователь_впр_сводный', 'таблица_еслимн_др', 'функция', 'визуализация_выдвижение_проверка', 'гипотеза', 'прогнозирование', 'обработка_больший_массив_данные', '1с', 'olap_куб', 'график_пятидневный_рабочий_неделя', '1', 'место_москва_сити', 'башня_ме

In [198]:
one = ['привет', 'как', 'дела', 'заработная', 'плата', 'я', 'программный', 'продукт']
trigrams[bigrams[one]]

['привет', 'как', 'дела', 'заработная', 'плата', 'я', 'программный_продукт']

In [ ]:
# for phrase, score in trigrams.find_phrases(descr_list).items():
#     print(phrase, score)

Обучение word2vec

In [148]:
from gensim.models import Word2Vec

In [199]:
model = Word2Vec(sentences=res, vector_size=100, window=5, min_count=5, workers=4, sg=0)

In [205]:
model.wv.most_similar('abc', topn = 10)

[('ям', 0.9288562536239624),
 ('gl', 0.9230470061302185),
 ('сканер', 0.916202187538147),
 ('excellent_communication', 0.9144453406333923),
 ('fpd', 0.9121344685554504),
 ('option', 0.9105832576751709),
 ('амортизация', 0.9104640483856201),
 ('order', 0.9103652834892273),
 ('пропуск', 0.9100919365882874),
 ('корректность_загрузка_данные', 0.9092960953712463)]

Обучение fastText

In [201]:
from gensim.models import FastText
model_ted = FastText(sentences=res, vector_size=100, window=5, min_count=5, workers=4,sg=1)

In [210]:
model_ted.wv.most_similar('gap_анализ', topn = 10)

[('swot_анализ', 0.9530337452888489),
 ('авс_анализ_xyz_анализ', 0.945347011089325),
 ('abc_анализ', 0.9376325607299805),
 ('авс_анализ', 0.9191548228263855),
 ('пост_анализ', 0.9152162075042725),
 ('abc_xyz_анализ', 0.9079082012176514),
 ('авс_xyz_анализ', 0.9068328738212585),
 ('анализ', 0.8584973812103271),
 ('постанализ', 0.8283380270004272),
 ('сценарный_анализ', 0.7996739745140076)]